# Multi-Modal Disease Prediction
Many reference kits  in the bio-medical domain focus on a single-model and single-modal solution. Exclusive reliance on a single method has some limitations, such as impairing the design of robust and accurate classifiers for complex datasets. To overcome these limitations, we provide this multi-modal disease prediction reference kit.

Multi-modal disease prediction is an Intel  optimized, end-to-end reference kit for fine-tuning and inference. This reference kit implements a multi-model and multi-modal solution that will help to predict diagnosis by using categorized contrast enhanced mammography data and radiologists’ notes.
 

## **Table of Contents**
- [Overview](#overview)
- [Validated Hardware Details](#validated-hardware-details)
- [Software Requirements](#software-requirements)
- [How it Works?](#how-it-works)
    - [Architecture](#architecture)
- [Get Started](#get-started)
- [Run Using Jupyter Lab](#run-using-jupyter-lab) 
- [Expected Output](#expected-output)
- [Result Visualization](#result-visualization)

<a id="overview"></a> 
## Overview
This reference kit demonstrates one possible reference implementation of a multi-model and multi-modal solution. While the vision workflow aims to train an image classifier that takes in contrast-enhanced spectral mammography (CESM) images, the natural language processing (NLP) workflow aims to train a document classifier that takes in annotation notes about a patient’s symptoms. Each pipeline creates a prediction for the diagnosis of breast cancer. In the end, weighted ensemble method is used to create a final prediction.

The goal is to minimize an expert’s involvement in categorizing samples as normal, benign, or malignant, by developing and optimizing a decision support system that automatically categorizes the CESM with the help of radiologist notes.

<a id="validated-hardware-details"></a> 
## Validated Hardware Details
There are workflow-specific hardware and software setup requirements depending on how the workflow is run. Bare metal development system and Docker image running locally have the same system requirements. 

| Recommended Hardware         | Precision  |
| ---------------------------- | ---------- |
| Intel® 4th Gen Xeon® Scalable Performance processors| FP32, BF16 |
| Intel® 1st, 2nd, 3rd, and 4th Gen Xeon® Scalable Performance processors| FP32 |

To execute the reference solution presented here, please use CPU for fine tuning. 

<a id="software-requirements"></a> 
## Software Requirements 
Linux OS (Ubuntu 22.04) is used in this reference solution. Make sure the following dependencies are installed, or if you followed the README file instructions for this notebook run the following cell.

1. `sudo apt update`
2. `sudo apt install -y build-essential gcc git libgl1-mesa-glx libglib2.0-0 python3-dev`
3. Python3.9, Pip/Conda and python3.9-venv


In [ ]:
import getpass
import os

password = getpass.getpass('Enter user password:\n')
command = 'sudo -S apt-get update'
os.popen(command, 'w').write(password+'\n')
command = 'sudo -S apt-get install -y build-essential gcc git libgl1-mesa-glx libglib2.0-0 python3-dev'
os.popen(command, 'w').write(password+'\n')

<a id="how-it-works"></a> 
## How It Works?

<a id="architecture"></a> 
### Architecture
![Use_case_flow](assets/e2e_flow_HLS_Disease_Prediction.png)
*Figure-1: Architecture of the reference kit* 

- Uses real-world CESM breast cancer datasets with “multi-modal and multi-model” approaches.
- Uses two domain toolkits (Intel® Transfer Learning Toolkit and Intel® Extension for Transformers), Intel® Neural Compressor, other libs/tools, Hugging Face model repo and APIs for [ResNet-50](https://huggingface.co/microsoft/resnet-50) and [ClinicalBert](https://huggingface.co/emilyalsentzer/Bio_ClinicalBERT) models. 
- The NLP reference Implementation component uses [HF Fine-tuning and Inference Optimization workload](https://github.com/intel/intel-extension-for-transformers/tree/main/workflows/hf_finetuning_and_inference_nlp), which is optimized for document classification. This NLP workload employs Intel® Neural Compressor and other libraries/tools, and utilizes Hugging Face model repository and APIs for ClinicalBert models. The ClinicalBert model, which is pretrained with a Masked-Language-Modeling task on a large corpus of English language from MIMIC-III data, is fine-tuned with the CESM breast cancer annotation dataset to generate a new BERT model.
- The Vision reference Implementation component uses [TLT-based vision workload](https://github.com/IntelAI/transfer-learning), which is optimized for image fine-tuning and inference. This workload utilizes Intel® Transfer Learning Tool and tfhub's ResNet-50 model to fine-tune a new convolutional neural network model with subtracted CESM image dataset. The images are preprocessed by using domain expert-defined segmented regions to reduce redundancies during training.
- Predicts diagnosis by using categorized contrast enhanced mammography images and radiologists’ notes separately. Weighted ensemble method is applied to results of sub-models to create the final prediction.

<a id="get-started"></a> 
## Get Started

### Download the Reference Kit Repository
If previously not done, from terminal create a working directory for the reference kit, clone the [Breast Cancer Prediction Reference Kit](https://github.com/intel/disease-prediction) repository into your working directory, launch Jupyter notebook and follow the README instructions to use the notebook before going any further.
```
git clone https://github.com/intel/disease-prediction.git brca_multimodal
cd brca_multimodal
jupyter notebook
```

### DataSet
The dataset is a collection of 2,006 high-resolution contrast-enhanced spectral mammography (CESM) images (1003 low energy images and 1003 subtracted CESM images) with annotations of 326 female patients. See Figure-1. Each patient has 8 images, 4 representing each side with two views (Top Down looking and Angled Top View) consisting of low energy and subtracted CESM images. Medical reports, written by radiologists, are provided for each case along with manual segmentation annotation for the abnormal findings in each image. As a preprocessing step, we segment the images based on the manual segmentation to get the region of interest and group annotation notes based on the subject and breast side. 

  ![CESM Images](assets/cesm_and_annotation.png)

*Figure-2: Samples of low energy and subtracted CESM images and Medical reports, written by radiologists from the Categorized contrast enhanced mammography dataset. [(Khaled, 2022)](https://www.nature.com/articles/s41597-022-01238-0)*

For more details of the dataset, visit the wikipage of the [CESM](https://wiki.cancerimagingarchive.net/pages/viewpage.action?pageId=109379611#109379611bcab02c187174a288dbcbf95d26179e8) and read [Categorized contrast enhanced mammography dataset for diagnostic and artificial intelligence research](https://www.nature.com/articles/s41597-022-01238-0).

#### Setting Up the Data
Use the link below to download the image datasets.

- [High-resolution Contrast-enhanced spectral mammography (CESM) images](https://faspex.cancerimagingarchive.net/aspera/faspex/external_deliveries/260?passcode=5335d2514638afdaf03237780dcdfec29edf4238#)

Copy the downloaded folders into the *data/CDD-CESM* directory.

**Note:** See this dataset's applicable license for terms and conditions. Intel Corporation does not own the rights to this dataset and does not confer any rights to it.

<a id="run-using-jupyter-lab"></a> 
# Run Using Jupyter Lab

### 1. Install Required Workflows and Preprocess the Data

> Make sure [High-resolution Contrast-enhanced spectral mammography (CESM) images](https://faspex.cancerimagingarchive.net/aspera/faspex/external_deliveries/260?passcode=5335d2514638afdaf03237780dcdfec29edf4238#) are downloaded and copied into the *data* directory.

This step involves the installation of the following components:

- HF Fine-tune & Inference Optimization workflow
- Transfer Learning based on TLT workflow
- Model Zoo for Intel®

Upon successful installation, the data is preprocessed.  


In [20]:
!git submodule update --init --recursive
!pip install docx2txt openpyxl

# Clone vision workflows
!pip install -r ${PWD}/vision_wf/workflows/disease_prediction/requirements.txt

# Clone hf-finetuning-inference-nlp-workflows
!pip install -r ${PWD}/hf_nlp/workflows/hf_finetuning_and_inference_nlp/requirements.txt

You should consider upgrading via the '/localdisk/aagalleg/demo_bc_test/disease-prediction/hls_env/bin/python -m pip install --upgrade pip' command.
  Using cached neural_compressor-2.0-py3-none-any.whl (1.1 MB)
  Attempting uninstall: neural-compressor
    Found existing installation: neural-compressor 2.1
    Uninstalling neural-compressor-2.1:
      Successfully uninstalled neural-compressor-2.1
You should consider upgrading via the '/localdisk/aagalleg/demo_bc_test/disease-prediction/hls_env/bin/python -m pip install --upgrade pip' command.
Looking in indexes: https://pypi.org/simple, https://download.pytorch.org/whl/cpu
  Using cached neural_compressor-2.1-py3-none-any.whl (1.2 MB)
  Attempting uninstall: neural-compressor
    Found existing installation: neural-compressor 2.0
    Uninstalling neural-compressor-2.0:
      Successfully uninstalled neural-compressor-2.0
You should consider upgrading via the '/localdisk/aagalleg/demo_bc_test/disease-prediction/hls_env/bin/python -m p


Do you agree to the terms and conditions? (y/n):
 y


Setup completed!
Terms and conditions agreed


E: Could not open lock file /var/lib/dpkg/lock-frontend - open (13: Permission denied)
E: Unable to acquire the dpkg frontend lock (/var/lib/dpkg/lock-frontend), are you root?


You should consider upgrading via the '/localdisk/aagalleg/demo_bc_test/disease-prediction/hls_env/bin/python -m pip install --upgrade pip' command.


In [21]:
#Preprocessing
#Execute this cell and scroll down. When reached the terms and conditions statement 
#the execution will pause until you agree (press y and Enter) or dissagree (press n and Enter).

import os
# Clone dataset_api
command = 'cd ${PWD}/intel-models/datasets/dataset_api/ && bash setup.sh'
os.popen(command, 'w').write(input("\nDo you agree to the terms and conditions? (y/n):\n"))

# run preprocessing steps for the dataset
!cd ${PWD}/intel-models/datasets/dataset_api/ && python dataset.py -n brca --download --preprocess -d ${PWD}/../../../data/

## Purpose of the Model Zoo

  - Demonstrate the AI workloads and deep learning models Intel has optimized and validated to run on Intel hardware
  - Show how to efficiently execute, train, and deploy Intel-optimized models
  - Make it easy to get started running Intel-optimized models on Intel hardware in the cloud or on bare metal

***DISCLAIMER: These scripts are not intended for benchmarking Intel platforms.
For any performance and/or benchmarking information on specific Intel platforms, visit https://www.intel.ai/blog.***

Intel is committed to the respect of human rights and avoiding complicity in human rights abuses, a policy reflected at https://www.intel.com/content/www/us/en/policy/policy-human-rights.html. Accordingly, by accessing the Intel material on this platform you agree that you will not use the material in a product or application that causes or contributes to a violation of an internationally recognized human right.

## License
The Model Zoo for Intel® Architecture 


Do you agree to the terms and conditions? (y/n):
 y


Terms and conditions agreed

File already exists in /localdisk/aagalleg/demo_bc_test/disease-prediction/intel-models/datasets/dataset_api/../../../data/Medical reports for cases .zip.
Please delete it and try again!.


File already exists in /localdisk/aagalleg/demo_bc_test/disease-prediction/intel-models/datasets/dataset_api/../../../data/Radiology manual annotations.xlsx.
Please delete it and try again!.


File already exists in /localdisk/aagalleg/demo_bc_test/disease-prediction/intel-models/datasets/dataset_api/../../../data/Radiology_hand_drawn_segmentations_v2.csv.
Please delete it and try again!.



E: Could not open lock file /var/lib/dpkg/lock-frontend - open (13: Permission denied)
E: Unable to acquire the dpkg frontend lock (/var/lib/dpkg/lock-frontend), are you root?


----- Starting the data preprocessing -----


You should consider upgrading via the '/localdisk/aagalleg/demo_bc_test/disease-prediction/hls_env/bin/python -m pip install --upgrade pip' command.


----- file is saved here : /localdisk/aagalleg/demo_bc_test/disease-prediction/intel-models/datasets/dataset_api/../../../data/annotation/annotation.csv
Intel(R) Extension for Scikit-learn* enabled (https://github.com/intel/scikit-learn-intelex)
Create folders for classified images
/localdisk/aagalleg/demo_bc_test/disease-prediction/intel-models/datasets/dataset_api/../../../data/
----- Classifying the images for data preprocessing -----
Classify Low energy images
Classify Subtracted energy images
/localdisk/aagalleg/demo_bc_test/disease-prediction/intel-models/datasets/dataset_api/../../../data/segmented_images/Normal/
Creating segmented images for "Normal" cases .........
Creating segmented images for "malignant" cases ..........
Creating segmented images for "Benign" cases ........


Once preprocessing has ended, you should have the following files and directories inside the `data/` directory:
```
data/ 
    |
    └──annotation/
        |── annotation.csv
        |── testing_data.csv
        └── training_data.csv
    └──CDD-CESM/
        |── Low energy images of CDD-CESM/
        └── Subtracted images of CDD-CESM/
    └──Medical reports for cases/
    └──segmented_images/
        |── Benign/
        |── Malignant/
        └── Normal/
    └──train_test_split_images
        |── test/
        └── train/
    └──vision_images
        |── Benign/
        |── Malignant/
        └── Normal/
    └──Radiology manual annotations.xlsx
    └──Radiology_hand_drawn_segmentations_v2.csv
    └──README.md
```

### 2. Model Building Process

To train the multi-model disease prediction, utilize the 'breast_cancer_prediction.py' script along with the arguments outlined in the 'disease_prediction_baremetal.yaml' configuration file, which has the following structure:

```
disease_prediction_baremetal.yaml
    
    |
    └──overwrite_training_testing_ids
    └──output_dir
    └──test_size
    └──write
    └──nlp
        |── finetune
        |── inference
        └── other parameters for HF fine-tune and inference optimization workflow
    └──vision
        |── finetune
        |── inference
        └── other parameters for HF fine-tune and inference optimization workflow
```

Descriptions of the previous parameters are as follow:

- overwrite_training_testing_ids: uses previously created train and test data sets
- output_dir: specifies the location of the output model and inference results
- test_size: sets the percentage of the test data set
- write: a container parameter that is set to false for bare metal
- nlp:
  - finetune: runs nlp fine-tuning
  - inference: runs nlp inference
  - additional parameters for the HF fine-tune and inference optimization workflow (more information available [here](https://github.com/intel/intel-extension-for-transformers/tree/main/workflows/hf_finetuning_and_inference_nlp/config))

- vision:
  - finetune: runs vision fine-tuning
  - inference: runs vision inference
  - additional parameters for the Vision: Transfer Learning Toolkit based on TLT workflow (more information available [here](https://github.com/IntelAI/transfer-learning/tree/f2e83f1614901d44d0fdd66f983de50551691676/workflows/disease_prediction))


To solely perform the fine-tuning process, set the '--inference' flag to 'False' and execute the following command:

In [2]:
!python src/breast_cancer_prediction.py --config_file configs/disease_prediction_baremetal.yaml --finetune=True --inference=False

### 3. Running Inference
After the models are trained and saved using the script from step 2, load the NLP and vision models using the inference option. This applies a weighted ensemble method to generate a final prediction. To only run inference, set the 'inference' parameter to true, the parameter 'finetune' to false and run the command provided in step 2.

> Alternatively, you can combine the training and inference processes into one execution by setting both the 'finetune' and 'inference' parameters to true and running the command provided in step 2.

<a id="expected-output"></a> 
## Expected Output
A successful execution of inference returns the confusion matrix of the sub-models and ensembled model, as shown in these example results: 
```
------ Confusion Matrix for Vision model ------
           Benign  Malignant  Normal  Precision
Benign       18.0     11.000   1.000      0.486
Malignant     5.0     32.000   0.000      0.615
Normal       14.0      9.000  25.000      0.962
Recall        0.6      0.865   0.521      0.652

------ Confusion Matrix for NLP model ---------
           Benign  Malignant  Normal  Precision
Benign     25.000      4.000     1.0      0.893
Malignant   3.000     34.000     0.0      0.895
Normal      0.000      0.000    48.0      0.980
Recall      0.833      0.919     1.0      0.930

------ Confusion Matrix for Ensemble --------
           Benign  Malignant  Normal  Precision
Benign     26.000      4.000     0.0      0.897
Malignant   3.000     34.000     0.0      0.895
Normal      0.000      0.000    48.0      1.000
Recall      0.867      0.919     1.0      0.939

```

<a id="result-visualization"></a> 
## Result Visualization
By utilizing the displayed widget, users can access a comprehensive overview that includes radiologists' annotation notes, corresponding subtracted CESM images, and ensemble predictions. Scroll down to see the selected results.

In [23]:
import importlib
import widget_manager
importlib.reload(widget_manager)

GridspecLayout(children=(Select(description='Patient ID:', layout=Layout(grid_area='widget001'), options=('Sel…

<module 'widget_manager' from '/localdisk/aagalleg/demo_bc_test/disease-prediction/widget_manager.py'>